# 第八章 聊天机器人

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
        <li>
        <span><a href="#二身份与上下文构建" data-toc-modified-id="二、身份与上下文构建">二、身份与上下文构建</a></span></li><li>
        <span><a href="#三订餐机器人" data-toc-modified-id="三、订餐机器人">三、订餐机器人</a></span>
        <ul class="toc-item">
            <li><span><a href="#31-创建json摘要" data-toc-modified-id="3.1 创建json摘要">3.1 创建json摘要</a></span></li>
        </ul>
        </li>
    </ul>
</div>

## 一、引言

使用一个大型语言模型的一个令人兴奋的事情是，我们可以用它来构建一个定制的聊天机器人 (Chatbot) ，只需要很少的工作量。在这一节中，我们将探索如何利用聊天的方式，与个性化（或专门针对特定任务或行为的）聊天机器人进行扩展对话。

像 ChatGPT 这样的聊天模型实际上是组装成以一系列消息作为输入，并返回一个模型生成的消息作为输出的。这种聊天格式原本的设计目标是简便多轮对话，但我们通过之前的学习可以知道，它对于不会涉及任何对话的**单轮任务**也同样有用。


## 二、身份与上下文构建

接下来，我们将定义两个辅助函数。

第一个方法已经陪伴了您一整个教程，即 ```get_completion``` ，其适用于单轮对话。我们将 Prompt 放入某种类似**用户消息**的对话框中。另一个称为 ```get_completion_from_messages``` ，传入一个消息列表。这些消息可以来自大量不同的**角色** (roles) ，我们会描述一下这些角色。

第一条消息中，我们以系统身份发送系统消息 (system message) ，它提供了一个总体的指示。系统消息则有助于设置助手的行为和角色，并作为对话的高级指示。你可以想象它在助手的耳边低语，引导它的回应，而用户不会注意到系统消息。因此，作为用户，如果你曾经使用过 ChatGPT，您可能从来不知道 ChatGPT 的系统消息是什么，这是有意为之的。系统消息的好处是为开发者提供了一种方法，在不让请求本身成为对话的一部分的情况下，引导助手并指导其回应。

在 ChatGPT 网页界面中，您的消息称为用户消息，而 ChatGPT 的消息称为助手消息。但在构建聊天机器人时，在发送了系统消息之后，您的角色可以仅作为用户 (user) ；也可以在用户和助手 (assistant) 之间交替，从而提供对话上下文。

In [37]:
from openai import OpenAI
api_key = 'sk-proj-BcPo4iWwrqzG6XpSQJdbT3BlbkFJ6hH061IxzGNMz1o4wCr0'
client = OpenAI(api_key=api_key)

def get_completion(prompt, model="gpt-3.5-turbo"):
# def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0

    )
    # print(response)
    return response.choices[0].message.content

 
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 控制模型输出的随机程度
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

现在让我们尝试在对话中使用这些消息。我们将使用上面的函数来获取从这些消息中得到的回答，同时，使用更高的温度 (temperature)（越高生成的越多样，更多内容见第七章）。

系统消息说，你是一个说话像莎士比亚的助手。这是我们向助手描述**它应该如何表现的方式**。然后，第一个用户消息是*给我讲个笑话*。接下来以助手身份给出回复是，*为什么鸡会过马路？* 最后发送用户消息是*我不知道*。

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [38]:
# 中文
messages =  [  
{'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},    
{'role':'user', 'content':'给我讲个笑话'},   
{'role':'assistant', 'content':'鸡为什么过马路'},   
{'role':'user', 'content':'我不知道'}  ]

In [45]:
response = get_completion_from_messages(messages, temperature=1.3)
print(response)

因为要展示自己的才艺，毕生中只有项目之一能表演得这么精彩——踏达坎岭，到超马路。



（注：上述例子中由于选定 temperature = 1，模型的回答会比较随机且迥异（不乏很有创意）。此处附上另一个回答：

让我用一首莎士比亚式的诗歌来回答你的问题：

当鸡之心欲往前，
马路之际是其选择。
驱车徐行而天晴，
鸣笛吹响伴交错。

问之何去何从也？
因大道之上未有征，
而鸡乃跃步前进，
其决策毋需犹豫。

鸡之智慧何可言，
道路孤独似乌漆。
然其勇气令人叹，
勇往直前没有退。

故鸡过马路何解？
忍受车流喧嚣之困厄。
因其鸣鸣悍然一跃，
成就夸夸骄人壁画。

所以笑话之妙处，
伴随鸡之勇气满溢。
笑谈人生不畏路，
有智有勇尽显妙。

希望这个莎士比亚风格的回答给你带来一些欢乐！

让我们看另一个例子。助手的消息是*你是一个友好的聊天机器人*，第一个用户消息是*嗨，我叫Isa*。我们想要得到第一个用户消息。

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [46]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'Hi, 我是Isa。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

你好，Isa！很高兴认识你。有什么我可以帮助你的吗？


让我们再试一个例子。系统消息是，你是一个友好的聊天机器人，第一个用户消息是，是的，你能提醒我我的名字是什么吗？

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [47]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然，你可以告诉我你的名字，然后我会记住并在我们的对话中使用它。你可以告诉我你的名字是什么吗？


如上所见，模型实际上并不知道我的名字。

因此，每次与语言模型的交互都互相独立，这意味着我们必须提供所有相关的消息，以便模型在当前对话中进行引用。如果想让模型引用或 “记住” 对话的早期部分，则必须在模型的输入中提供早期的交流。我们将其称为上下文 (context) 。尝试以下示例。

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

附上另一次回答：

*Your name is Isa! How could I forget?*

In [48]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然，你的名字是Isa。有什么其它问题我可以帮助你解决吗？


现在我们已经给模型提供了上下文，也就是之前的对话中提到的我的名字，然后我们会问同样的问题，也就是我的名字是什么。因为模型有了需要的全部上下文，所以它能够做出回应，就像我们在输入的消息列表中看到的一样。

## 三、订餐机器人

现在，我们构建一个 “订餐机器人”，我们需要它自动收集用户信息，接受比萨饼店的订单。

下面这个函数将收集我们的用户消息，以便我们可以避免像刚才一样手动输入。这个函数将从我们下面构建的用户界面中收集 Prompt ，然后将其附加到一个名为上下文( ```context``` )的列表中，并在每次调用模型时使用该上下文。模型的响应也会添加到上下文中，所以用户消息和模型消息都被添加到上下文中，上下文逐渐变长。这样，模型就有了需要的信息来确定下一步要做什么。

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

现在，我们将设置并运行这个 UI 来显示订单机器人。初始的上下文包含了包含菜单的系统消息，在每次调用时都会使用。此后随着对话进行，上下文也会不断增长。

In [ ]:
!pip install panel

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

运行结果可交互，请见下文中文版。

### 3.1 创建JSON摘要

此处我们另外要求模型创建一个 JSON 摘要，方便我们发送给订单系统。

因此我们需要在上下文的基础上追加另一个系统消息，作为另一条指示 (instruction) 。我们说*创建一个刚刚订单的 JSON 摘要，列出每个项目的价格，字段应包括 1）披萨，包括尺寸，2）配料列表，3）饮料列表，4）辅菜列表，包括尺寸，最后是总价格*。此处也可以定义为用户消息，不一定是系统消息。

请注意，这里我们使用了一个较低的温度，因为对于这些类型的任务，我们希望输出相对可预测。

In [49]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

# 中文
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

{
    "pizza": {
        "size": "medium",
        "type": "cheese",
        "price": 9.25
    },
    "toppings": [],
    "drinks": [
        {
            "type": "cola",
            "size": "medium",
            "price": 2.00
        }
    ],
    "sides": [],
    "total_price": 11.25
}


C:\Users\Administrator.DESKTOP-RJ8HC2Q\AppData\Local\Temp\ipykernel_24976\3067100770.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [50]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'创建上一个食品订单的 json 摘要。\
逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
    "披萨": {
        "名称": "茄子披萨",
        "大小": "大号",
        "价格": 11.95
    },
    "配料列表": [
        {
            "名称": "蘑菇",
            "价格": 1.50
        },
        {
            "名称": "香肠",
            "价格": 3.00
        }
    ],
    "饮料列表": [
        {
            "名称": "可乐",
            "大小": "大号",
            "价格": 3.00
        }
    ],
    "配菜列表": [],
    "总价": 19.45
}


现在，我们已经建立了自己的订餐聊天机器人。请随意自定义并修改系统消息，以更改聊天机器人的行为，并使其扮演不同的角色，拥有不同的知识。

附：下图展示了订餐机器人一次完整的对话流程：
![image.png](../../figures/Chatbot-pizza-cn.png)

In [101]:
from openai import OpenAIapi_key = 'sk-proj-BcPo4iWwrqzG6XpSQJdbT3BlbkFJ6hH061IxzGNMz1o4wCr0'
client = OpenAI(api_key=api_key)
def get_completion(prompt, model="gpt-3.5-turbo"):
# def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
                     "text": "没问题！你想要哪种披萨呢？选择有意式辣香肠、芝士和茄子披萨哦。每种披萨都有大、中、小三个尺寸。你想要什么口味和尺寸呢？"
                    }
                  ]
                },
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": "我想买芝士披萨,中号的"
                    }
                  ]
                },
                {
                  "role": "assistant",
                  "content": [
                    {
                      "type": "text",
                      "text": "好的，芝士披萨中号一个，加起来是9.25元。\n\n需要加点什么配料吗？有奶酪、蘑菇、香肠、加拿大熏肉、AI酱，还有辣椒哦。每种配料的价格都写在菜单上，你可以随意搭配！"
                    }
                  ]
                },
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": "我需要加配料,加拿大熏肠,AI酱.."
                    }
                  ]
                },
                {
                  "role": "assistant",
                  "content": [
                    {
                      "type": "text",
                      "text": "好的，加拿大熏肉和AI酱已经添加到你的中号芝士披萨上了。额外配料的费用是3.50元和1.50元，总共是5.00元。这样你的披萨总价格是14.25元。\n\n还需要添加什么吗？或者要点什么其他的菜品或饮料吗？"
                    }
                  ]
                },
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": "我需要大号的可乐"
                    }
                  ]
                },
                {
                  "role": "assistant",
                  "content": [
                    {
                      "type": "text",
                      "text": "好的，一大号的可乐，加上3.00元。现在总价格是17.25元。\n\n你的订单如下：\n- 中号芝士披萨 9.25元\n- 加拿大熏肉 3.50元\n- AI酱 1.50元\n- 大号可乐 3.00元\n\n还需要加点什么吗？还是说你准备好确认订单了？另外，你是要自取还是需要外送呢？如果是外送，请告诉我你的地址。"
                    }
                  ]
                },
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": "自取"
                    }
                  ]
                },
                {
                  "role": "assistant",
                  "content": [
                    {
                      "type": "text",
                      "text": "好的，自取就方便多了。\n\n你的订单总价格是17.25元：\n- 中号芝士披萨 9.25元\n- 加拿大熏肉 3.50元\n- AI酱 1.50元\n- 大号可乐 3.00元\n\n订单已经确认！期待你的到来，祝你愉快！"
                    }
                  ]
                }
        ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content
      ]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [108]:
from openai import OpenAI
api_key = 'sk-proj-BcPo4iWwrqzG6XpSQJdbT3BlbkFJ6hH061IxzGNMz1o4wCr0'
client = OpenAI(api_key=api_key)

def get_completion(prompt):
    response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "你是订餐机器人，为披萨餐厅自动收集订单信息。\n你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。\n最后需要询问是否自取或外送，如果是外送，你要询问地址。\n最后告诉顾客订单总金额，并送上祝福。\n\n请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。\n你的回应应该以简短、非常随意和友好的风格呈现。\n\n菜单包括：\n\n菜品：\n意式辣香肠披萨（大、中、小） 12.95、10.00、7.00\n芝士披萨（大、中、小） 10.95、9.25、6.50\n茄子披萨（大、中、小） 11.95、9.75、6.75\n薯条（大、小） 4.50、3.50\n希腊沙拉 7.25\n\n配料：\n奶酪 2.00\n蘑菇 1.50\n香肠 3.00\n加拿大熏肉 3.50\nAI酱 1.50\n辣椒 1.00\n\n饮料：\n可乐（大、中、小） 3.00、2.00、1.00\n雪碧（大、中、小） 3.00、2.00、1.00\n瓶装水 5.00"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "你好！我们这里有各种美味的披萨。你想要哪种披萨？有意式辣香肠、芝士和茄子披萨，每种都有大、中、小三个尺寸喔。请告诉我你想要哪种披萨和尺寸吧。"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "我想买披萨"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "没问题！你想要哪种披萨呢？选择有意式辣香肠、芝士和茄子披萨哦。每种披萨都有大、中、小三个尺寸。你想要什么口味和尺寸呢？"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "我想买芝士披萨,中号的"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "好的，芝士披萨中号一个，加起来是9.25元。\n\n需要加点什么配料吗？有奶酪、蘑菇、香肠、加拿大熏肉、AI酱，还有辣椒哦。每种配料的价格都写在菜单上，你可以随意搭配！"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "我需要加配料,加拿大熏肠,AI酱.."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "好的，加拿大熏肉和AI酱已经添加到你的中号芝士披萨上了。额外配料的费用是3.50元和1.50元，总共是5.00元。这样你的披萨总价格是14.25元。\n\n还需要添加什么吗？或者要点什么其他的菜品或饮料吗？"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "我需要大号的可乐"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "好的，一大号的可乐，加上3.00元。现在总价格是17.25元。\n\n你的订单如下：\n- 中号芝士披萨 9.25元\n- 加拿大熏肉 3.50元\n- AI酱 1.50元\n- 大号可乐 3.00元\n\n还需要加点什么吗？还是说你准备好确认订单了？另外，你是要自取还是需要外送呢？如果是外送，请告诉我你的地址。"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "自取"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "好的，自取就方便多了。\n\n你的订单总价格是17.25元：\n- 中号芝士披萨 9.25元\n- 加拿大熏肉 3.50元\n- AI酱 1.50元\n- 大号可乐 3.00元\n\n订单已经确认！期待你的到来，祝你愉快！"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
    return response.choices[0].message.content

get_completion("我临时有事,想改为外送!")

'没问题，改为外送的话，请告诉我你的地址吧，我们会尽快为你送达！'